In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim 

def make_batch():
    """
  extracts words of each sentence in the sentences, then appends a list
  of the first n-1 words to the input_batch and the target to the target_batch.

  Returns: 
  - input_batch: a list of inputs, the i-th input is a list of first n-1 words
                  of the i-th sentence in the sentences
  - target_batch: a list of targets, the i-th target is the last word of the 
                  i-th sentence in the sentences
  """

    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch


In [ ]:
class NNLM(nn.Module):
    """
  implementation of Neural Probabilistic Language Model.

  """
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step * m, n_hidden, bias=False)
        self.d = nn.Parameter(torch.ones(n_hidden))
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step * m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))

    def forward(self, X):
        """
      performs a forward operation on the on the class instance.

      Inputs:
      - X: a tensor of shape [batch_size, n_steps, m]
      """
        X = self.C(X)  # X : [batch_size, n_steps, m]
        X = X.view(-1, n_step * m)  # [batch_size, n_steps * m]
        tanh = torch.tanh(self.d + self.H(X))  # [batch_size, n_hidden]
        output = self.b + self.W(X) + self.U(tanh)  # [batch_size, n_class]
        return output

In [ ]:
if __name__ == '__main__':
    n_step = 2  # number of steps, n-1 in paper
    n_hidden = 2  # number of hidden size, h in paper
    m = 2 # embedding size, m in paper

    sentences = ["I like dog", "I love coffee", "I hate milk"]

    word_list = " ".join(sentences).split()  # getting all of the 
                                             # words in the sentences
    word_list = list(set(word_list))  # getting the unique words
    word_dict = {w: i for i, w in enumerate(word_list)}  # mapping words to 
                                                         # numbers
    number_dict = {i: w for i, w in enumerate(word_list)}  # mapping numbers to 
                                                           # words
    n_class = len(word_list)  # n_class is the vocabulary size

    model = NNLM()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    input_batch, target_batch = make_batch()
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)

    # Training the model
    for epoch in range(5000):
        optimizer.zero_grad()
        output = model(input_batch)

        # output : [batch_size, n_class]
        # target_batch : [batch_size]

        loss = criterion(output, target_batch)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =',
                  '{:.6f}'.format(loss))
            
        loss.backward()
        optimizer.step()

    # Predict
    predictions = model(input_batch).data.max(1, keepdim=True)[1]

    # Test
    print([sen.split()[:2] for sen in sentences], '->',
          [number_dict[n.item()] for n in predictions.squeeze()])



Epoch: 1000 cost = 0.074641
Epoch: 2000 cost = 0.012813
Epoch: 3000 cost = 0.004531
Epoch: 4000 cost = 0.002038
Epoch: 5000 cost = 0.001022
[['I', 'like'], ['I', 'love'], ['I', 'hate']] -> ['dog', 'coffef', 'milk']
